# Assignment Explanation

In this assignment, you'll create a Python script to perform a sentiment analysis of the Twitter activity of various news oulets, and to present your findings visually.

Your final output should provide a visualized summary of the sentiments expressed in the last 100 Tweets sent out by the following news organizations: __BBC, CBS, CNN, Fox, and New York times__.

Your final Jupyter notebook must:

* Pull last 100 tweets from each outlet.
* Perform a sentiment analysis with the compound, positive, neutral, and negative scoring for each tweet. 
* Pull into a DataFrame the tweet's source acount, its text, its date, and its compound, positive, neutral, and negative sentiment scores.
* Export the data in the DataFrame into a CSV file.
* Save PNG images for each plot.

As final considerations:

* Use the Matplotlib and Seaborn libraries.
* Include a written description of three observable trends based on the data. 
* Include proper labeling of your plots, including plot titles (with date of analysis) and axes labels.
* Include an exported markdown version of your Notebook called  `README.md` in your GitHub repository.  


Hints, requirements, and considerations:

* You may find it helpful to organize your code in function(s), then call them.
* If you're not yet familiar with creating functions in Python, here is a tutorial you may wish to consult: [https://www.tutorialspoint.com/python/python_functions.htm](https://www.tutorialspoint.com/python/python_functions.htm).



## Copyright

Coding Boot Camp (C) 2017. All Rights Reserved.

# NewsVADERPy: News Mood Analysis

- Trend 1
- Trend 2
- Trend 3

In [49]:
# ----------------------------------------------------------------------
# Step 1: Import necessary modules and environment
# ----------------------------------------------------------------------

# import libraries
import matplotlib as plt
import pandas as pd
import seaborn as sns

# This file contains all Twitter-related actions, so no need to import here
import tweetParser as tp

In [57]:
# ----------------------------------------------------------------------
# Step 2: Call API, get tweets, and parse tweets into a dataframe+CSV
# ----------------------------------------------------------------------

# create list of target news organizations' Twitter handles
targetNewsOrg_list = ["BBC","CNN"]

# define number of tweets we want to pull from each org
numTweets = 100

# break into increments of 10
numCycles = int(round(numTweets/10))

# create dict to store dictionaries generated during analysis
completeResults_df = parseTweets(targetNewsOrg_list, numCycles)

# rearrange columns to be more sensible
completeResults_df = completeResults_df[["handle","compound","positive","negative","neutral","date"]]
completeResults_df

,handle,compound,positive,negative,neutral,date
0,BBC,0.4019,0.184,0.089,0.726,Sun Mar 04 20:11:35 +0000 2018
1,BBC,0.8468,0.386,0.000,0.614,Sun Mar 04 19:00:05 +0000 2018
2,BBC,0.4215,0.118,0.000,0.882,Sun Mar 04 18:59:39 +0000 2018
3,BBC,-0.3182,0.000,0.099,0.901,Sun Mar 04 18:00:09 +0000 2018
4,BBC,0.4588,0.111,0.000,0.889,Sun Mar 04 16:57:46 +0000 2018
5,BBC,0.6369,0.189,0.000,0.811,Sun Mar 04 16:30:12 +0000 2018
6,BBC,0.3382,0.130,0.000,0.870,Sun Mar 04 15:30:29 +0000 2018
7,BBC,0.1779,0.124,0.000,0.876,Sun Mar 04 15:00:06 +0000 2018
8,BBC,0.0498,0.180,0.170,0.650,Sun Mar 04 14:03:03 +0000 2018
9,BBC,0.8020,0.298,0.000,0.702,Sun Mar 04 13:46:48 +0000 2018


The first plot will be and/or feature the following:

- Be a scatter plot of sentiments of the last 100 tweets sent out by each news organization, ranging from -1.0 to 1.0, where a score of 0 expresses a neutral sentiment, -1 the most negative sentiment possible, and +1 the most positive sentiment possible.
- Each plot point will reflect the compound sentiment of a tweet.
- Sort each plot point by its relative timestamp.


In [ ]:
# ----------------------------------------------------------------------
# Step 4: Generate first plot: scatterplot of last 100 tweets showing 
# compound sentiment and sorted by relative timestamp
# ----------------------------------------------------------------------

bar plot visualizing the overall sentiments of the last 100 tweets from each organization. 
For this plot, you will again aggregate the compound sentiments analyzed by VADER.



In [ ]:
# ----------------------------------------------------------------------
# Step 5: Generate second plot: bar plot showing overall compound 
# sentiment in the last 100 tweets
# ----------------------------------------------------------------------